<a href="https://colab.research.google.com/github/biz-HK/colab_test.ipynb/blob/main/colab_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI画像検査システム - クラウド環境テスト
Google Colabで実行可能なテストコード

## 1. 環境セットアップ

In [1]:
# 必要なライブラリのインストール
!pip install torch torchvision opencv-python-headless pillow numpy -q
print("パッケージインストール完了")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.6 MB/s eta 0:00:00
パッケージインストール完了


In [2]:
# インポートテスト
import torch
import torchvision
import cv2
import numpy as np
from PIL import Image
import torch.nn as nn
import torchvision.transforms as T

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
print(f"OpenCV version: {cv2.__version__}")

PyTorch version: 2.6.0+cu124
CUDA available: False
OpenCV version: 4.12.0


## 2. モデル定義（元のコードから）

In [3]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel,self).__init__()
        self.Encoder = nn.Sequential(self.create_convblock(3,16),     #256
                                     nn.MaxPool2d((2,2)),
                                     self.create_convblock(16,32),    #128
                                     nn.MaxPool2d((2,2)),
                                     self.create_convblock(32,64),    #64
                                     nn.MaxPool2d((2,2)),
                                     self.create_convblock(64,128),   #32
                                     nn.MaxPool2d((2,2)),
                                     self.create_convblock(128,256),  #16
                                     nn.MaxPool2d((2,2)),
                                     self.create_convblock(256,512),  #8
                                    )
        self.Decoder = nn.Sequential(self.create_deconvblock(512,256), #16
                                     self.create_convblock(256,256),
                                     self.create_deconvblock(256,128), #32
                                     self.create_convblock(128,128),
                                     self.create_deconvblock(128,64),  #64
                                     self.create_convblock(64,64),
                                     self.create_deconvblock(64,32),   #128
                                     self.create_convblock(32,32),
                                     self.create_deconvblock(32,16),   #256
                                     self.create_convblock(16,16),
                                    )
        self.last_layer = nn.Conv2d(16,3,1,1)

    def create_convblock(self,i_fn,o_fn):
        conv_block = nn.Sequential(nn.Conv2d(i_fn,o_fn,3,1,1),
                                   nn.BatchNorm2d(o_fn),
                                   nn.ReLU(),
                                   nn.Conv2d(o_fn,o_fn,3,1,1),
                                   nn.BatchNorm2d(o_fn),
                                   nn.ReLU()
                                  )
        return conv_block

    def create_deconvblock(self,i_fn , o_fn):
        deconv_block = nn.Sequential(nn.ConvTranspose2d(i_fn, o_fn, kernel_size=2, stride=2),
                                      nn.BatchNorm2d(o_fn),
                                      nn.ReLU(),
                                     )
        return deconv_block

    def forward(self,x):
        x = self.Encoder(x)
        x = self.Decoder(x)
        x = self.last_layer(x)
        return x

print("モデル定義完了")

モデル定義完了


## 3. モデル作成とパラメータ確認

In [4]:
# モデル作成
model = CustomModel()

# GPU使用可能なら移動
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# モデルパラメータ数
total_params = sum(p.numel() for p in model.parameters())
print(f"モデルパラメータ数: {total_params:,}")
print(f"使用デバイス: {device}")

モデルパラメータ数: 6,996,979
使用デバイス: cpu


## 4. ダミー画像でテスト

In [5]:
# ダミー画像生成
dummy_image = np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8)

# 前処理
preprocess = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
])

# PIL画像に変換してテンソル化
pil_image = Image.fromarray(dummy_image)
input_tensor = preprocess(pil_image).unsqueeze(0).to(device)

print(f"入力テンソルサイズ: {input_tensor.shape}")

# 推論実行
model.eval()
with torch.no_grad():
    output = model(input_tensor)

print(f"出力テンソルサイズ: {output.shape}")
print("推論成功！")

入力テンソルサイズ: torch.Size([1, 3, 256, 256])
出力テンソルサイズ: torch.Size([1, 3, 256, 256])
推論成功！


## 5. 異常検知シミュレーション

In [6]:
def calculate_reconstruction_error(original, reconstructed):
    """復元誤差を計算（MSE）"""
    mse = torch.mean((original - reconstructed) ** 2)
    return mse.item()

# 正常画像シミュレーション（一様な画像）
normal_image = np.ones((256, 256, 3), dtype=np.uint8) * 128
normal_tensor = preprocess(Image.fromarray(normal_image)).unsqueeze(0).to(device)

# 異常画像シミュレーション（ランダムノイズ）
anomaly_image = np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8)
anomaly_tensor = preprocess(Image.fromarray(anomaly_image)).unsqueeze(0).to(device)

# 推論と誤差計算
with torch.no_grad():
    normal_output = model(normal_tensor)
    anomaly_output = model(anomaly_tensor)

    normal_error = calculate_reconstruction_error(normal_tensor, normal_output)
    anomaly_error = calculate_reconstruction_error(anomaly_tensor, anomaly_output)

print(f"正常画像の復元誤差: {normal_error:.6f}")
print(f"異常画像の復元誤差: {anomaly_error:.6f}")
print(f"\n※学習前のモデルなので、実際の異常検知性能は期待できません")
print(f"  学習後は異常画像の誤差が大きくなります")

正常画像の復元誤差: 0.173182
異常画像の復元誤差: 0.253664

※学習前のモデルなので、実際の異常検知性能は期待できません
  学習後は異常画像の誤差が大きくなります


## 6. メモリ使用量チェック

In [7]:
import psutil
import os

# CPU メモリ
process = psutil.Process(os.getpid())
mem_info = process.memory_info()
print(f"CPU メモリ使用量: {mem_info.rss / 1024 / 1024:.1f} MB")

# GPU メモリ（利用可能な場合）
if torch.cuda.is_available():
    print(f"GPU メモリ使用量: {torch.cuda.memory_allocated() / 1024 / 1024:.1f} MB")
    print(f"GPU メモリ予約量: {torch.cuda.memory_reserved() / 1024 / 1024:.1f} MB")

CPU メモリ使用量: 643.1 MB


## 7. ベンチマークテスト

In [ ]:
import time

# バッチサイズごとの処理速度測定
batch_sizes = [1, 4, 8, 16]
results = []

for batch_size in batch_sizes:
    # ダミーバッチ作成
    batch = torch.randn(batch_size, 3, 256, 256).to(device)

    # ウォームアップ
    for _ in range(3):
        _ = model(batch)

    # 計測
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    start = time.time()

    for _ in range(10):
        _ = model(batch)

    torch.cuda.synchronize() if torch.cuda.is_available() else None
    elapsed = time.time() - start

    fps = (batch_size * 10) / elapsed
    results.append((batch_size, fps))
    print(f"バッチサイズ {batch_size:2d}: {fps:6.2f} FPS")

print(f"\n最適バッチサイズ: {max(results, key=lambda x: x[1])[0]}")

バッチサイズ  1:   4.77 FPS
バッチサイズ  4:   4.74 FPS
バッチサイズ  8:   4.31 FPS


## まとめ
このノートブックをGoogle Colabにアップロードして実行することで：
- PyTorch/OpenCVの動作確認
- モデルアーキテクチャの検証
- GPU利用可能性の確認
- パフォーマンス測定

が可能です。